In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.optimize import minimize
import pyreadr
import os
import time
from scipy.stats import norm
import copy
import warnings
import utils
import regression_fit
import pickle
from importlib import reload
reload(utils)
reload(regression_fit)
from utils import *
from regression_fit import *

home = os.getcwd()

## Dataset

In [24]:
GroupA = pyreadr.read_r(home + '\\Other_data\\GroupA.rda')
GroupA_df = pd.DataFrame(list(GroupA.values())[0])

GroupB = pyreadr.read_r(home + '\\Other_data\\GroupB.rda')
GroupB_df = pd.DataFrame(list(GroupB.values())[0])

GroupC = pyreadr.read_r(home + '\\Other_data\\GroupC.rda')
GroupC_df = pd.DataFrame(list(GroupC.values())[0])

GroupD = pyreadr.read_r(home + '\\Other_data\\GroupD.rda')
GroupD_df = pd.DataFrame(list(GroupD.values())[0])

GroupE = pyreadr.read_r(home + '\\Other_data\\GroupE.rda')
GroupE_df = pd.DataFrame(list(GroupE.values())[0])

GroupF = pyreadr.read_r(home + '\\Other_data\\GroupF.rda')
GroupF_df = pd.DataFrame(list(GroupF.values())[0])

GroupG = pyreadr.read_r(home + '\\Other_data\\GroupG.rda')
GroupG_df = pd.DataFrame(list(GroupG.values())[0])

GroupH = pyreadr.read_r(home + '\\Other_data\\GroupH.rda')
GroupH_df = pd.DataFrame(list(GroupH.values())[0])

GroupJ = pyreadr.read_r(home + '\\Other_data\\GroupJ.rda')
GroupJ_df = pd.DataFrame(list(GroupJ.values())[0])

GroupH = pyreadr.read_r(home + '\\Other_data\\GroupH.rda')
GroupH_df = pd.DataFrame(list(GroupH.values())[0])

GroupJ = pyreadr.read_r(home + '\\Other_data\\GroupJ.rda')
GroupJ_df = pd.DataFrame(list(GroupJ.values())[0])

GroupK = pyreadr.read_r(home + '\\Other_data\\GroupK.rda')
GroupK_df = pd.DataFrame(list(GroupK.values())[0])

GroupL = pyreadr.read_r(home + '\\Other_data\\GroupL.rda')
GroupL_df = pd.DataFrame(list(GroupL.values())[0])

GroupM = pyreadr.read_r(home + '\\Other_data\\GroupM.rda')
GroupM_df = pd.DataFrame(list(GroupM.values())[0])

GroupN = pyreadr.read_r(home + '\\Other_data\\GroupN.rda')
GroupN_df = pd.DataFrame(list(GroupN.values())[0])

GroupP = pyreadr.read_r(home + '\\Other_data\\GroupP.rda')
GroupP_df = pd.DataFrame(list(GroupP.values())[0])

In [27]:
regions_df = [GroupA_df, GroupB_df, GroupC_df, GroupD_df, GroupE_df, GroupF_df, GroupG_df, GroupH_df, GroupJ_df, GroupK_df,
             GroupL_df, GroupM_df, GroupN_df, GroupP_df]

# Indexes of the test set corresponding to the beginning of 2020 and of 2021 (needed to compute the metrics which are split by year)
idx_2020 = 17516
idx_2021 = 35104

In [29]:
# Adding columns y0_96, y0_336, diff_96, diff_336 to the dataset of each region
for df in regions_df:   
    df = add_autoregressive_features(df)

regions = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P']
train_set = {}
test_set = {}

# The dataset of each region is split in train and test set
for (reg, df) in zip(regions, regions_df):
    # Train set
    train_set[reg] = df[720:87440] 
    #Test set
    test_set[reg] = df[87440:]

# Linear Regression Model 
The linear regression model in regression_fit.py is trained on the train set of each region

Target Variable: 7d differenced Net-Load 

In [30]:
for (reg, df) in zip(regions, regions_df):
    St_train = regression_feature_matrix(train_set[reg])
    model = sm.OLS(train_set[reg]["diff_336"], St_train)
    results = model.fit()   
    params = results.params

    St = regression_feature_matrix(df)

    df['y0_diff_OLS'] = np.array(df['y0_336']) + St@params
    df['res_diff_OLS'] = np.array(df['node']) - np.array(df['y0_diff_OLS'])

for (reg, df) in zip(regions, regions_df):
    # Train set
    train_set[reg] = df[720:87440] 
    #Test set
    test_set[reg] = df[87440:]

## Evaluation of the linear regression model on the test set

In [32]:
idx_2020 = 17516
idx_2021 = 35104

y_mat = np.zeros((len(test_set['A']),len(regions)))
for i,reg in enumerate(regions):
    y_mat[:,i] = test_set[reg]['node']
    
yhat_seasonality_diff_OLS = np.zeros((len(test_set['A']),len(regions)))
    
for i, reg in enumerate(regions):
    yhat_seasonality_diff_OLS[:,i] = test_set[reg]['y0_diff_OLS']
    

print("\nnRMSE linear model on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_seasonality_diff_OLS[:idx_2020]))
print("nRMSE linear model on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_seasonality_diff_OLS[idx_2020:idx_2021]))
print("nRMSE linear model on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_seasonality_diff_OLS[idx_2021:]))

print("\nnMAE linear model on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_seasonality_diff_OLS[:idx_2020]))
print("nMAE linear model on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_seasonality_diff_OLS[idx_2020:idx_2021]))
print("nMAE linear model on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_seasonality_diff_OLS[idx_2021:]))



nRMSE linear model on 2019 test set:  0.3210659939224211
nRMSE linear model on 2020 test set:  0.3655281882300815
nRMSE linear model on 2021 test set:  0.3730800094368767

nMAE linear model on 2019 test set:  0.2899933816913386
nMAE linear model on 2020 test set:  0.3376767823569824
nMAE linear model on 2021 test set:  0.34967470434870546


# Kalman Filter on the linear regression model

## Static Kalman Filter

Static Kalman Filter is trained on the train set of each region (target variable is still the 7d differenced net-load)

### 48 hours delay in receiving data is considered

In [33]:
idx_2020 = 17516
idx_2021 = 35104
kf_static_diff = {}

for reg in regions:
    print(reg)
    St_train = regression_feature_matrix(train_set[reg])
    kf = Kalman_Filter(GAM=False)
    kf.fit(St_train, train_set[reg]['diff_336'])
    kf_static_diff[reg] = kf

y_Kalman_static_diff = {}

for reg in regions:
    St_test = regression_feature_matrix(test_set[reg])
    # Target variable for predictions are the 7d differences
    df_kf = test_set[reg][['targetTime','diff_336']]
    y_Kalman_static_diff[reg] = {}
    y_Kalman_static_diff[reg]['2019'], _ = kf_static_diff[reg].predict(St_test[:idx_2020], df_kf.iloc[:idx_2020], delay=True) 
    y_Kalman_static_diff[reg]['2020'], _ = kf_static_diff[reg].predict(St_test[idx_2020:idx_2021], df_kf.iloc[idx_2020:idx_2021], delay=True) 
    y_Kalman_static_diff[reg]['2021'], _ = kf_static_diff[reg].predict(St_test[idx_2021:], df_kf.iloc[idx_2021:], delay=True) 
    
    # The value of 7d before is added back
    y_Kalman_static_diff[reg]['2019'] += test_set[reg]['y0_336'].iloc[:idx_2020]
    y_Kalman_static_diff[reg]['2020'] += test_set[reg]['y0_336'].iloc[idx_2020:idx_2021] 
    y_Kalman_static_diff[reg]['2021'] += test_set[reg]['y0_336'].iloc[idx_2021:]

A
B
C
D
E
F
G
H
J
K
L
M
N
P


## Evaluation of the Static KF on the test set

In [34]:
idx_2020 = 17516
idx_2021 = 35104

y_mat = np.zeros((len(test_set['A']),len(regions)))
for i,reg in enumerate(regions):
    y_mat[:,i] = test_set[reg]['node']
    
yhat_staticKF_diff_mat = np.zeros((len(test_set['A']),len(regions)))


for i,reg in enumerate(regions):
    yhat_staticKF_diff_mat[:idx_2020,i] = y_Kalman_static_diff[reg]['2019']
    yhat_staticKF_diff_mat[idx_2020:idx_2021,i] = y_Kalman_static_diff[reg]['2020']
    yhat_staticKF_diff_mat[idx_2021:,i] = y_Kalman_static_diff[reg]['2021']


print("\nnRMSE linear model on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_seasonality_diff_OLS[:idx_2020]))
print("nRMSE linear model on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_seasonality_diff_OLS[idx_2020:idx_2021]))
print("nRMSE linear model on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_seasonality_diff_OLS[idx_2021:]))

print("\nnMAE linear model on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_seasonality_diff_OLS[:idx_2020]))
print("nMAE linear model on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_seasonality_diff_OLS[idx_2020:idx_2021]))
print("nMAE linear model on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_seasonality_diff_OLS[idx_2021:]))

print("\n\nnRMSE Static KF on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_staticKF_diff_mat[:idx_2020]))
print("nRMSE Static KF on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_staticKF_diff_mat[idx_2020:idx_2021]))
print("nRMSE Static KF on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_staticKF_diff_mat[idx_2021:]))

print("\nnMAE Static KF on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_staticKF_diff_mat[:idx_2020]))
print("nMAE Static KF on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_staticKF_diff_mat[idx_2020:idx_2021]))
print("nMAE Static KF on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_staticKF_diff_mat[idx_2021:]))



nRMSE linear model on 2019 test set:  0.3210659939224211
nRMSE linear model on 2020 test set:  0.3655281882300815
nRMSE linear model on 2021 test set:  0.3730800094368767

nMAE linear model on 2019 test set:  0.2899933816913386
nMAE linear model on 2020 test set:  0.3376767823569824
nMAE linear model on 2021 test set:  0.34967470434870546


nRMSE Static KF on 2019 test set:  0.32093671911562077
nRMSE Static KF on 2020 test set:  0.3548937292164037
nRMSE Static KF on 2021 test set:  0.36206766483762304

nMAE Static KF on 2019 test set:  0.2894053981226853
nMAE Static KF on 2020 test set:  0.32385440022716405
nMAE Static KF on 2021 test set:  0.3378300236983406


# Dynamic Kalman Filter

### 48 hours delay in receiving data is considered
Target variable: 7d differenced net-load

In [35]:
idx_2020 = 17516
idx_2021 = 35104

kf_dynamic_diff = {}

q_list = [1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15, 1e-16, 1e-17, 1e-18]
#sigma_list = [30,25,22,20,15,13,12,11,10,9,8,7, 5, 4.5, 4, 3.5, 3, 2.5, 2, 1.5, 1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.1, 0.08, 0.05]


for reg in regions:
    print(reg)
    St_train = regression_feature_matrix(train_set[reg])
    
    std_static = np.std(kf_static_diff[reg].theta_mat[1000:],axis=0)
    
    optim = Q_optimization(St_train, train_set[reg][["targetTime", "diff_336"]],GAM=False)
    Q, sigma, P, theta1 = optim.grid_search_reduced_likelihood(q_list=q_list, std_static=std_static)
    kf = Kalman_Filter(GAM=False, Q=Q, sigma=sigma, P=P, theta1=theta1)
    kf.fit(St_train, train_set[reg]['diff_336'])
    kf_dynamic_diff[reg] = kf
    
y_Kalman_dynamic_diff = {}

for reg in regions:
    St_test = regression_feature_matrix(test_set[reg])
    # Target variable for predictions are the 7d differences
    df_kf = test_set[reg][['targetTime','diff_336']]
    y_Kalman_dynamic_diff[reg] = {}
    y_Kalman_dynamic_diff[reg]['2019'], _ = kf_dynamic_diff[reg].predict(St_test[:idx_2020], df_kf.iloc[:idx_2020], delay=True) 
    y_Kalman_dynamic_diff[reg]['2020'], _ = kf_dynamic_diff[reg].predict(St_test[idx_2020:idx_2021], df_kf.iloc[idx_2020:idx_2021], delay=True) 
    y_Kalman_dynamic_diff[reg]['2021'], _ = kf_dynamic_diff[reg].predict(St_test[idx_2021:], df_kf.iloc[idx_2021:], delay=True)
    
    # The value of 7d before is added back
    y_Kalman_dynamic_diff[reg]['2019'] += test_set[reg]['y0_336'].iloc[:idx_2020]
    y_Kalman_dynamic_diff[reg]['2020'] += test_set[reg]['y0_336'].iloc[idx_2020:idx_2021] 
    y_Kalman_dynamic_diff[reg]['2021'] += test_set[reg]['y0_336'].iloc[idx_2021:]

A
1e-10
-485177.4430397677
1e-11
-478669.46424540656
1e-12
-474985.10351807915
1e-13
-472107.1205804807
1e-14
-468342.571766466
1e-15
-467044.7209839821
1e-16
-466847.27708004654
1e-17
-467112.4970620406
1e-18
-467717.60099707375
Optimal values q and sigma:  1.6310348837729667e-12 127.71197609358985
B
1e-10
-467772.64213982713
1e-11
-461889.65973727754
1e-12
-456977.8355193931
1e-13
-452908.3974177182
1e-14
-449516.7487992976
1e-15
-448442.90347887354
1e-16
-448194.41637506813
1e-17
-448210.504309099
1e-18
-448501.4800050854
Optimal values q and sigma:  1.056797883668117e-12 102.80067527346876
C
1e-10
-435408.32044473675
1e-11
-428317.85666508676
1e-12
-427095.4130453489
1e-13
-422170.0133804249
1e-14
-418500.3733233612
1e-15
-417749.2151213994
1e-16
-417794.3042253129
1e-17
-418079.2707749869
1e-18
-418658.7991400517
Optimal values q and sigma:  5.262282031350064e-12 72.54158828803008
D
1e-10
-417522.59024313756
1e-11
-407677.59105254646
1e-12
-399315.9871894867
1e-13
-393425.99264484

## Evaluation of the Dynamic KF on the test set

In [37]:
y_mat = np.zeros((len(test_set['A']),len(regions)))
for i,reg in enumerate(regions):
    y_mat[:,i] = test_set[reg]['node']
    
yhat_staticKF_diff_mat = np.zeros((len(test_set['A']),len(regions)))
yhat_dynamicKF_diff_mat = np.zeros((len(test_set['A']),len(regions)))

for i,reg in enumerate(regions):
    yhat_staticKF_diff_mat[:idx_2020,i] = y_Kalman_static_diff[reg]['2019']
    yhat_staticKF_diff_mat[idx_2020:idx_2021,i] = y_Kalman_static_diff[reg]['2020']
    yhat_staticKF_diff_mat[idx_2021:,i] = y_Kalman_static_diff[reg]['2021']
    yhat_dynamicKF_diff_mat[:idx_2020,i] = y_Kalman_dynamic_diff[reg]['2019']
    yhat_dynamicKF_diff_mat[idx_2020:idx_2021,i] = y_Kalman_dynamic_diff[reg]['2020']
    yhat_dynamicKF_diff_mat[idx_2021:,i] = y_Kalman_dynamic_diff[reg]['2021']


In [38]:
idx_2020 = 17516
idx_2021 = 35104

print("\nnRMSE linear model on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_seasonality_diff_OLS[:idx_2020]))
print("nRMSE linear model on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_seasonality_diff_OLS[idx_2020:idx_2021]))
print("nRMSE linear model on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_seasonality_diff_OLS[idx_2021:]))

print("\nnMAE linear model on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_seasonality_diff_OLS[:idx_2020]))
print("nMAE linear model on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_seasonality_diff_OLS[idx_2020:idx_2021]))
print("nMAE linear model on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_seasonality_diff_OLS[idx_2021:]))

print("\n\nnRMSE Static KF on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_staticKF_diff_mat[:idx_2020]))
print("nRMSE Static KF on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_staticKF_diff_mat[idx_2020:idx_2021]))
print("nRMSE Static KF on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_staticKF_diff_mat[idx_2021:]))

print("\nnMAE Static KF on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_staticKF_diff_mat[:idx_2020]))
print("nMAE Static KF on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_staticKF_diff_mat[idx_2020:idx_2021]))
print("nMAE Static KF on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_staticKF_diff_mat[idx_2021:]))

print("\n\nnRMSE Dynamic KF on 2019 test set: ", nRMSE(y_mat[:idx_2020], yhat_dynamicKF_diff_mat[:idx_2020]))
print("nRMSE Dynamic KF on 2020 test set: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_dynamicKF_diff_mat[idx_2020:idx_2021]))
print("nRMSE Dynamic KF on 2021 test set: ", nRMSE(y_mat[idx_2021:], yhat_dynamicKF_diff_mat[idx_2021:]))

print("\nnMAE Dynamic KF on 2019 test set: ", nMAE(y_mat[:idx_2020], yhat_dynamicKF_diff_mat[:idx_2020]))
print("nMAE Dynamic KF on 2020 test set: ", nMAE(y_mat[idx_2020:idx_2021], yhat_dynamicKF_diff_mat[idx_2020:idx_2021]))
print("nMAE Dynamic KF on 2021 test set: ", nMAE(y_mat[idx_2021:], yhat_dynamicKF_diff_mat[idx_2021:]))



nRMSE linear model on 2019 test set:  0.3210659939224211
nRMSE linear model on 2020 test set:  0.3655281882300815
nRMSE linear model on 2021 test set:  0.3730800094368767

nMAE linear model on 2019 test set:  0.2899933816913386
nMAE linear model on 2020 test set:  0.3376767823569824
nMAE linear model on 2021 test set:  0.34967470434870546


nRMSE Static KF on 2019 test set:  0.32093671911562077
nRMSE Static KF on 2020 test set:  0.3548937292164037
nRMSE Static KF on 2021 test set:  0.36206766483762304

nMAE Static KF on 2019 test set:  0.2894053981226853
nMAE Static KF on 2020 test set:  0.32385440022716405
nMAE Static KF on 2021 test set:  0.3378300236983406


nRMSE Dynamic KF on 2019 test set:  0.3197862426635117
nRMSE Dynamic KF on 2020 test set:  0.3445159327990319
nRMSE Dynamic KF on 2021 test set:  0.35892274353904197

nMAE Dynamic KF on 2019 test set:  0.2889279102536277
nMAE Dynamic KF on 2020 test set:  0.31307389141984737
nMAE Dynamic KF on 2021 test set:  0.3338451693037776